In [319]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import urllib

import numpy as np
import tensorflow as tf

In [320]:
import os
import pandas as pd
ROOT_PATH = "C:\\Users\\micha\\Downloads\\"

In [373]:
num_of_genes = 500
num_of_samples = 2000

In [374]:
#order of samples in this file defines sample id
sample_columns = os.path.join(ROOT_PATH, "Sample_Columns.txt")
sample_file = open(sample_columns,'r')
sample_names = []
for line in sample_file:
    parts = line.split('\t')
    for part in parts:
        sample_name = part.rstrip()
        sample_names.append(sample_name)
#sample_names = sample_file.read().split()
#remove Name and Description entries
del(sample_names[0])
del(sample_names[0])
print(sample_names[0])
sample_to_id = {}
sample_id = 1
for name in sample_names:
    #name = name.rstrip()
    sample_to_id[name] = sample_id
    #print(name + " " + str(sample_id))
    sample_id = sample_id + 1

GTEX-1117F-0226-SM-5GZZ7


In [375]:
#do not need this section of code for current version
gene_column = os.path.join(ROOT_PATH, "Gene_Column.txt")
gene_file = open(gene_column,'r')
gene_names = []
gene_counter = 1
for line in gene_file:
    if (gene_counter < 51):
        gene_names.append(line.rstrip())
        #gene_names.append(str(gene_counter))
        gene_counter = gene_counter + 1
gene_names.append("Tissue_ID")

In [376]:
sample_to_tissue = {}
tissue_to_id = {}
tissues = []
sample_tissue_path = os.path.join(ROOT_PATH, "Sample_to_Tissue.txt")
sample_tissue_file = open(sample_tissue_path,'r')
tissue_id = 1
for line in sample_tissue_file:
    parts = line.split('\t')
    sample_name = parts[0].rstrip()
    if (sample_name in sample_to_id.keys()):
        sample_id = sample_to_id[sample_name]
        tissue = parts[1].rstrip()
        tissue = tissue.replace(',', ' ')

        #tissues.append(tissue)
        if (tissue not in tissue_to_id.keys()):
            tissue_to_id[tissue] = tissue_id
            tissues.append(tissue)
            tissue_id = tissue_id + 1
        sample_to_tissue[sample_id] = tissue_to_id[tissue]

In [377]:
# the header in the training and test files needs to be a comma-separated
#list of number of entries, number of features, followed by list of all possible classes/categories 
header_string = str(num_of_samples) + "," + str(num_of_genes) +"," + ",".join(tissues)

In [378]:
num_of_classes = len(tissues)
print(str(num_of_classes))

53


In [379]:
tpm_data = os.path.join(ROOT_PATH, "GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_tpm.gct")
#use first 50 genes
reader = pd.read_table(tpm_data, sep = "\t",skiprows = [0,1], nrows=num_of_genes)

In [380]:
TRAINING = "training7.csv"
TEST = "test7.csv"

In [381]:
f = open(TRAINING, "w")
f.write(header_string + "\n")
i = 2
sample_id = 1
#use first 500 samples
while i < 2 + num_of_samples:
    list = []
    tissue_id = sample_to_tissue[sample_id] 
    #each column contains expression values for each gene in a sample
    for index, column in reader.iterrows() :
        list.append(str(round(column[i],0)))
    #print(len(list))
    #append tissue_id to end of line
    list.append(str(tissue_id))
    line = ",".join(list) + "\n"
    f.write(line)
    sample_id =  sample_id + 2
    i = i + 2
f.close()

In [382]:
f_test = open(TEST, "w")
f_test.write(header_string + "\n")

i = 3
sample_id = 2
while i < 2 + num_of_samples:
    list = []
    tissue_id = sample_to_tissue[sample_id] 
    #each column contains expression values for each gene in a sample
    for index, column in reader.iterrows() :
        list.append(str(round(column[i],0)))
    #print(len(list))
    #append tissue_id to end of line
    list.append(str(tissue_id))
    line = ",".join(list) + "\n"
    f_test.write(line)
    sample_id = sample_id + 2
    i = i + 2
f_test.close()

In [383]:
# Load datasets.
training_set = tf.contrib.learn.datasets.base.load_csv_with_header(
      filename=TRAINING,
      target_dtype=np.int,
      features_dtype=np.float32)
test_set = tf.contrib.learn.datasets.base.load_csv_with_header(
      filename=TEST,
      target_dtype=np.int,
      features_dtype=np.float32)

In [384]:
# Specify that all features have real-value data
feature_columns = [tf.feature_column.numeric_column("x", shape=[num_of_genes])]

In [385]:
# Build 3 layer DNN with 10, 20, 10 units respectively.
classifier = tf.estimator.DNNClassifier(feature_columns=feature_columns,
                                          hidden_units=[10, 20, 10],
                                          n_classes=54,
                                          model_dir="")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\micha\\AppData\\Local\\Temp\\tmpfuvu65rb', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}


In [386]:
# Define the training inputs
train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": np.array(training_set.data)},
      y=np.array(training_set.target),
      num_epochs=None,
      shuffle=True)

In [387]:
# Train model.
classifier.train(input_fn=train_input_fn, steps=2000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\micha\AppData\Local\Temp\tmpfuvu65rb\model.ckpt.
INFO:tensorflow:loss = 10185.3, step = 1
INFO:tensorflow:global_step/sec: 492.253
INFO:tensorflow:loss = 149.688, step = 101 (0.219 sec)
INFO:tensorflow:global_step/sec: 492.255
INFO:tensorflow:loss = 87.5721, step = 201 (0.188 sec)
INFO:tensorflow:global_step/sec: 457.094
INFO:tensorflow:loss = 154.117, step = 301 (0.219 sec)
INFO:tensorflow:global_step/sec: 533.276
INFO:tensorflow:loss = 54.6652, step = 401 (0.188 sec)
INFO:tensorflow:global_step/sec: 492.257
INFO:tensorflow:loss = 113.056, step = 501 (0.203 sec)
INFO:tensorflow:global_step/sec: 492.253
INFO:tensorflow:loss = 77.9297, step = 601 (0.203 sec)
INFO:tensorflow:global_step/sec: 533.279
INFO:tensorflow:loss = 62.7609, step = 701 (0.203 sec)
INFO:tensorflow:global_step/sec: 492.253
INFO:tensorflow:loss = 112.234, step = 801 (0.203 sec)
INFO:tensorflow:global_step/sec: 492.258
IN

In [388]:
# Define the test inputs
test_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": np.array(test_set.data)},
      y=np.array(test_set.target),
      num_epochs=1,
      shuffle=False)

In [389]:
# Evaluate accuracy.
accuracy_score = classifier.evaluate(input_fn=test_input_fn)["accuracy"]

print("\nTest Accuracy: {0:f}\n".format(accuracy_score))

INFO:tensorflow:Starting evaluation at 2017-10-22-20:21:10
INFO:tensorflow:Restoring parameters from C:\Users\micha\AppData\Local\Temp\tmpfuvu65rb\model.ckpt-2000
INFO:tensorflow:Finished evaluation at 2017-10-22-20:21:10
INFO:tensorflow:Saving dict for global step 2000: accuracy = 0.8415, average_loss = 0.524371, global_step = 2000, loss = 65.5463

Test Accuracy: 0.841500



In [ ]:
# Classify one new tissue sample.

new_samples = np.array(
#test example with 100 genes - need to update if number of included genes change
    [[0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,322.0,30.0,9.0,7.0,2306.0,9.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,1.0,6.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,62.0,12.0,1.0,0.0,8.0,46.0,0.0,0.0,82.0,18.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,235.0,16.0,1.0,0.0,24.0,3.0,2.0,29.0,9.0,61.0,28.0,17.0,0.0,62.0]], dtype=np.float32)
predict_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": new_samples},
      num_epochs=1,
      shuffle=False)

predictions = classifier.predict(input_fn=predict_input_fn)
#dir(predictions)
d = next(predictions)
print(type(d))
print(d.keys())
probs= d['probabilities']
max_p_index = np.argmax(probs)
print(max_p_index)
print(tissues[max_p_index])